# 1-Librairies

In [93]:
# Dataset
import kagglehub

# Standard libraries
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm as notebook_tqdm
from PIL import Image
import glob

# tensorflow libraries
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.applications import VGG19
# keras libraries
from keras.optimizers import Adam

# scikit-learn libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Suppression des warnings
import warnings
warnings.filterwarnings("ignore")

# 2-Dataset

In [94]:
# Téléchargement du dataset depuis Kaggle
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("Chemin d'accés du dossier du Dataset:", path)

# Classification des images de tumeurs cérébrales
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
# Affichage des noms de classes
print("Class names:", class_names)

# Convertir les noms de classes en binaire
# Ici, nous considérons 'notumor' comme 'Sain' et les autres comme 'Tumeur'
binary_class_names = ['Sain' if name == 'notumor' else 'Tumeur' for name in class_names]

# Affichage des noms de classes binaires
binary_class_names = dict(zip(class_names, binary_class_names))
print("Binary class names:", binary_class_names)

Chemin d'accés du dossier du Dataset: C:\Users\mgraz\.cache\kagglehub\datasets\masoudnickparvar\brain-tumor-mri-dataset\versions\1
Class names: ['glioma', 'meningioma', 'notumor', 'pituitary']
Binary class names: {'glioma': 'Tumeur', 'meningioma': 'Tumeur', 'notumor': 'Sain', 'pituitary': 'Tumeur'}


## 2.1-Dataset Entrainement

In [95]:
# Création de train_df à partir du dossier Training
train_images = []
train_binary_class_names = []

# Parcours des images dans le dossier Training
train_dir = os.path.join(path, 'Training')
for category in binary_class_names:
	category_path = os.path.join(train_dir, category)
	for img_name in os.listdir(category_path):
		train_images.append(os.path.join(category_path, img_name))
		train_binary_class_names.append(binary_class_names[category])
# Affichage du nombre d'images dans le dataset d'entraînement
print("Nombre d'images dans le dataset d'entraînement:", len(train_images))
# Création du DataFrame train_df
train_df = pd.DataFrame({"image_path": train_images, "label": train_binary_class_names})

X_train = train_df["image_path"].values
y_train = train_df["label"].values

Nombre d'images dans le dataset d'entraînement: 5712


## 2.2-Dataset Test

In [96]:
# Création de test_df à partir du dossier Testing
test_images = []
test_binary_class_names = []

# Parcours des images dans le dossier Testing
test_dir = os.path.join(path, 'Testing')
for category in class_names:
	category_path = os.path.join(test_dir, category)
	for img_name in os.listdir(category_path):
		test_images.append(os.path.join(category_path, img_name))
		test_binary_class_names.append(binary_class_names[category])

# Affichage du nombre d'images dans le dataset d'entraînement
print("Nombre d'images dans le dataset d'entraînement:", len(test_images))
# Création du DataFrame train_df
test_df = pd.DataFrame({"image_path": test_images, "label": test_binary_class_names})

X_test = test_df["image_path"].values
y_test = test_df["label"].values

Nombre d'images dans le dataset d'entraînement: 1311


## 2.3-Dataset de Validation

In [97]:
# Creation d'un dataset de validation
X_train, X_val, y_train, y_val = train_test_split(
    train_df["image_path"].to_numpy(),
    train_df["label"].to_numpy(),
    train_size=0.7,  # 80% des données pour l'entraînement
    test_size=0.3,   # 20% des données pour la validation
    random_state=42,
    stratify=train_df["label"],
)
val_df = pd.DataFrame({'image_path': X_val, 'label': y_val})

# Affichage du nombre d'images par classes dans le dataset de validation
print("Nombre d'images par classes dans le dataset de test:", val_df['label'].value_counts())

Nombre d'images par classes dans le dataset de test: label
Tumeur    1235
Sain       479
Name: count, dtype: int64


# 3-Preprocessing

## 3.1-Paramêtres

In [98]:
# Creation des hyperparamètres
def create_params():
    return {
        'batch_size': 32,  # Taille du batch
        'image_size': (224, 224),  # Taille des images pour VGG19
        'input_shape': (224, 224, 3),  # Forme d'entrée pour les images RGB
        'epochs': 20,  # Nombre d'époques pour l'entraînement
        'learning_rate': 0.0001,  # Taux d'apprentissage pour l'optimiseur
    }

params = create_params()

## 3.2-ImageDataGenerator

In [99]:
# Création d'un ImageDataGenerator pour l'augmentation des données d'entraînement 
def create_datagen():
    """
    Crée un ImageDataGenerator pour l'augmentation des données d'entraînement.
    """
    return tf.keras.preprocessing.image.ImageDataGenerator(
        horizontal_flip=True,         # Retourner les images horizontalement
        rotation_range=10,            # Rotation aléatoire des images
        zoom_range=0.1,               # Zoom aléatoire des images
        width_shift_range=0.1,        # Décalage horizontal aléatoire
        height_shift_range=0.1,       # Décalage vertical aléatoire
        rescale=1/255,                # Normalisation des pixels entre 0 et 1
        brightness_range=(0.8, 1.2)   # Changement de la luminosité des images
    )
datagen = create_datagen()

## 3.3- Equilibrage du dataset d'entrainement

In [100]:
# Affichage du nombre d'images dans le DataFrame train_df
print("Nombre d'images dans le train_df:", len(train_df))
# Affichage du nombre d'images par classes dans le dataset d'entraînement
print("Nombre d'images par classes dans le dataset de train:", train_df['label'].value_counts())

Nombre d'images dans le train_df: 5712
Nombre d'images par classes dans le dataset de train: label
Tumeur    4117
Sain      1595
Name: count, dtype: int64


### 3.3.1-Séparation du label en 2 sous-classes

In [101]:
# Filtrage des images 'Sain' dans le DataFrame train_df
notumor_train_df = train_df[train_df['label'] == 'Sain']
# Filtrage des images 'Tumeur' dans le DataFrame train_df
# Ici, nous considérons 'Tumeur' comme toutes les autres classes sauf 'Sain
tumor_train_df = train_df[train_df['label'] == 'Tumeur']

# Affichage du nombre d'images dans le DataFrame filtré
print("Nombre d'images 'Sain':", len(notumor_train_df))
print("Nombre d'images 'Tumeur':", len(tumor_train_df))

Nombre d'images 'Sain': 1595
Nombre d'images 'Tumeur': 4117


### 3.3.2-Augmentation de la classe minoritaire

In [102]:
# Appliquer l'augmentation des données uniquement sur les images 'Sain'
notumor_train_df_datagen_images = []
# 
for x in notumor_train_df['image_path'].values:
    # Charger l'image, la redimensionner et la convertir en array
    img = tf.keras.utils.load_img(x, target_size=params['image_size'])
    img_array = tf.keras.utils.img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)  # nécessaire pour ImageDataGenerator
    for i, batch in enumerate(datagen.flow(img_array, batch_size=1)):
        notumor_train_df_datagen_images.append(batch[0])
        if i >= 2:  # 1 augmentations par image
            break
X_class1_aug = np.array(notumor_train_df_datagen_images)

# Affichage du nombre d'images augmentées et du DataFrame d'entrainement
print("Nombre d'images 'Sain' dans le train_df:", len(notumor_train_df))
print("Nombre d'images augmentées 'Sain':", len(X_class1_aug))
print("Nombre d'images dans le DataFrame d'entrainement après augmentation:", len(tumor_train_df) + len(X_class1_aug))

Nombre d'images 'Sain' dans le train_df: 1595
Nombre d'images augmentées 'Sain': 4785
Nombre d'images dans le DataFrame d'entrainement après augmentation: 8902


### 3.3.3-Création du dataset d'entrainement augmenté

In [103]:
# Création du DataFrame d'entrainement avec les images augmentées
augmented_notumor_paths = notumor_train_df_datagen_images
augmented_notumor_labels = ['Sain'] * len(augmented_notumor_paths)

# On garde les images "Tumeur" telles quelles
tumor_paths = list(tumor_train_df['image_path'].values)
tumor_labels = ['Tumeur'] * len(tumor_paths)

# On concatène les deux listes
train_augmented_images = tumor_paths + augmented_notumor_paths
train_aug_binary_class_names = tumor_labels + augmented_notumor_labels

# Affichage du nombre d'images dans le dataset d'entraînement
print("Nombre d'images dans le dataset d'entraînement augmenté:", len(train_augmented_images))

# Création du DataFrame train_df_augmented
train_df_augmented = pd.DataFrame({"image_path": train_augmented_images, "label": train_aug_binary_class_names})
X_train = train_df_augmented["image_path"].values
y_train = train_df_augmented["label"].values

# Sauvegarde du dataset augmenté
output_path = "./train_df_augmented"
notumor_path = os.path.join(output_path, "Sain")
tumor_path = os.path.join(output_path, "Tumeur")

os.makedirs(notumor_path, exist_ok=True)
os.makedirs(tumor_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

for i, img_array in enumerate(augmented_notumor_paths):
    img = Image.fromarray(img_array.astype(np.uint8))
    img.save(os.path.join(notumor_path, f"notumor{i}.png"))

for i, img_path in enumerate(tumor_paths):
    img = Image.open(img_path)
    img = img.resize(params['image_size'])
    img.save(os.path.join(tumor_path, f"tumor{i}.png"))

print("Nombre d'images par classes dans le dataset d'entraînement augmenté:", train_df_augmented['label'].value_counts())

Nombre d'images dans le dataset d'entraînement augmenté: 8902
Nombre d'images par classes dans le dataset d'entraînement augmenté: label
Sain      4785
Tumeur    4117
Name: count, dtype: int64


## 3.4-Création des dataset ImageDataGenerator

In [104]:
# Correction: Utiliser uniquement les chemins d'accès (strings) pour le DataFrame d'entraînement augmenté

# Récupérer tous les chemins d'images et labels depuis le dossier output_path
sain_image_paths = sorted(glob.glob(os.path.join(output_path, "Sain", "*.png")))
tumeur_image_paths = sorted(glob.glob(os.path.join(output_path, "Tumeur", "*.png")))

train_augmented_filepaths = sain_image_paths + tumeur_image_paths
train_augmented_labels = ["Sain"] * len(sain_image_paths) + ["Tumeur"] * len(tumeur_image_paths)

train_augmented_df = pd.DataFrame({
    "image_path": train_augmented_filepaths,
    "label": train_augmented_labels
})

# Création du générateur pour les données d'entraînement augmenté
train_augmented_datagen = datagen.flow_from_dataframe(
    train_augmented_df,
    x_col='image_path',
    y_col='label',
    target_size=params['image_size'],
    batch_size=params['batch_size'],
    class_mode='binary'
)

# Création du générateur pour les données de validation
val_datagen = datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label',
    target_size=params['image_size'],
    batch_size=params['batch_size'],
    class_mode='binary'
)

# Création du générateur pour les données de test
test_datagen = datagen.flow_from_dataframe(
    test_df,
    x_col='image_path',
    y_col='label',
    target_size=params['image_size'],
    batch_size=params['batch_size'],
    class_mode='binary',
    shuffle=False
)

Found 8902 validated image filenames belonging to 2 classes.
Found 1714 validated image filenames belonging to 2 classes.
Found 1311 validated image filenames belonging to 2 classes.


# 4-CNN (Réseau Convolutif)

## 4.1-Création du modèle 

In [105]:
# Création du modèle CNN avec VGG19 pré-entraîné
base_model = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=params['input_shape'])
# Gel des couches du modèle de base
base_model.trainable = False

# Affichage de l'architecture du modèle de base
print(f"Nombre de couches: {len(base_model.layers)}")
print(f"Forme de sortie: {base_model.output_shape}")
print(f"Couches gelées: {not base_model.trainable}")

# Construction du modèle complet
print("Construction de l'architecture complète...")

# Ajouter nouvelle couche de classification
x = base_model.output 
x = tf.keras.layers.GlobalAveragePooling2D()(x) # Pooling global pour réduire les dimensions
x = tf.keras.layers.Dense(256, activation='relu')(x)  # Couche dense avec 256 neurones et activation ReLU
x = tf.keras.layers.Dropout(0.2)(x) # Couche de dropout pour la régularisation
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x) # Couche de classification finale

# Création du modèle final
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

print("Modèle créé avec succès !")

# Affichage de l'architecture du modèle
print(" Architecture du modèle:")
model.summary()

# Configuration de l'optimiseur et des métriques
print("=== PHASE 1: TRANSFER LEARNING (couches gelées) ===")

# Compilation du modèle
model.compile(
    optimizer=Adam(learning_rate=params['learning_rate']),  # Utilisation de l'optimiseur Adam avec un taux d'apprentissage réduit
    loss='binary_crossentropy',  # Utilisation de 'binary_crossentropy' pour la classification binaire
    metrics=['accuracy', 'Precision', 'Recall']  # Ajout de métriques supplémentaires
)

print("Modèle compilé !")

Nombre de couches: 22
Forme de sortie: (None, 7, 7, 512)
Couches gelées: True
Construction de l'architecture complète...
Modèle créé avec succès !
 Architecture du modèle:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 20,155,969 (76.89 MB)

 Trainable params: 131,585 (514.00 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

=== PHASE 1: TRANSFER LEARNING (couches gelées) ===
Modèle compilé !


## 4.2-Entrainement du modele

In [ ]:
# Création des callbacks
def create_callbacks():
    callbacks_list = [

        # Sauvegarde du meilleur modèle
        callbacks.ModelCheckpoint(
            'model_cnn_US1_v2.keras',
            monitor='loss',
            save_best_only=True,
            verbose=1
        ),

        # Early stopping pour éviter l'overfitting
        callbacks.EarlyStopping(
            monitor='loss',
            patience=3,
            restore_best_weights=True,
        )
    ]
    return callbacks_list

# Création des callbacks
callbacks_list = create_callbacks()

# Entraînement du modèle
print("=== Début de l'entraînement ===")

history = model.fit(
    train_augmented_datagen,
    validation_data=val_datagen,
    epochs=params['epochs'],
    callbacks=callbacks_list,
)

print("=== Entraînement terminé ===")

=== Début de l'entraînement ===
Epoch 1/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - Precision: 0.9459 - Recall: 0.9195 - accuracy: 0.9392 - loss: 0.3305
Epoch 1: loss improved from inf to 0.18418, saving model to model_cnn_US1_v2.keras
279/279 ━━━━━━━━━━━━━━━━━━━━ 970s 3s/step - Precision: 0.9461 - Recall: 0.9197 - accuracy: 0.9394 - loss: 0.3299 - val_Precision: 0.7205 - val_Recall: 1.0000 - val_accuracy: 0.7205 - val_loss: 1.1859
Epoch 2/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - Precision: 1.0000 - Recall: 0.9999 - accuracy: 1.0000 - loss: 0.0308
Epoch 2: loss improved from 0.18418 to 0.02352, saving model to model_cnn_US1_v2.keras
279/279 ━━━━━━━━━━━━━━━━━━━━ 958s 3s/step - Precision: 1.0000 - Recall: 0.9999 - accuracy: 1.0000 - loss: 0.0307 - val_Precision: 0.7205 - val_Recall: 1.0000 - val_accuracy: 0.7205 - val_loss: 1.6287
Epoch 3/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - Precision: 1.0000 - Recall: 1.0000 - accuracy: 1.0000 - loss: 0.0110
Epoch 3: loss improved from 0.02

In [ ]:
history.history.keys()

## 4.3-Evaluation du modèle

In [ ]:
# Evaluation du modèle
loss, accuracy, precision, recall = model.evaluate(test_datagen)
print(f"Test Loss: {loss:0.5f}")
print(f"Test Accuracy: {accuracy:0.5f}")
print(f"Test Precision: {precision:0.5f}")
print(f"Test Recall: {recall:0.5f}")

## 4.4-Visualisation du modèle

In [ ]:
_, ax = plt.subplots(ncols=2, figsize=(15, 6))

# Graphique accuracy training/validation
ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Model Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Accuracy')
ax[0].legend(['Entrainement', 'Validation'])
ax[0].grid(alpha=0.2)

# Graphique loss training/validation
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Model Loss')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].legend(['Entrainement', 'Validation'])
ax[1].grid(alpha=0.2)

plt.show()

In [ ]:
def test_model(model, test_datagen):
    # Charger les poids du meilleur modèle sauvegardé
    model.load_weights('model_cnn_US1_v2.keras')
    predictions = model.predict(test_datagen)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_datagen.classes
    test_acc = 100 * np.sum(predicted_classes == true_classes) / len(true_classes)

    print(f"Test Accuracy: {test_acc:.2f}%\n")

    print("Classification Report:\n")
    print(classification_report(true_classes, predicted_classes, target_names=binary_class_names))

print("\nLoading best model for testing...")
try:
    model
except NameError:
    print("Erreur : 'model' n'est pas défini. Veuillez exécuter la cellule où le modèle est créé (cellule 21).")
else:
    test_model(model, test_datagen)

# 5-Prédiction

In [ ]:
# Visualisation des prédictions sur un échantillon d'images du dataset de test
def multi_to_binary(y_class_pred, positif_classes=['glioma', 'meningioma', 'pituitary']):
    return ['Tumeur' if y in positif_classes else 'Sain' for y in y_class_pred]

# Prédire les classes sur le jeu de test avec le générateur
y_pred_probs = model.predict(test_datagen)
y_pred = [binary_class_names[np.argmax(probs)] for probs in y_pred_probs]
# Convertir les classes en "Tumeur"/"Sain"
y_binary_pred = multi_to_binary(y_pred)
# Convertir les classes en "Tumeur"/"Sain" pour les labels réels
y_true = [binary_class_names[i] for i in test_datagen.classes]
y_binary_true = multi_to_binary(y_true)

# Definir la fonction predictions
def predictions(model, test_generator, num_samples=20, figsize=(20, 20)):
    plt.figure(figsize=figsize)
    rows = int(np.floor(np.sqrt(num_samples)))
    cols = int(np.ceil(num_samples / rows))

    # Lot d'images et labels du test generator
    images, true_labels_encoded = next(test_generator)
    # Convertir les indices en noms de classes
    true_class_names = [class_names[np.argmax(label)] for label in true_labels_encoded]
    # Convertir en labels binaires
    true_labels = multi_to_binary(true_class_names, positif_classes=['glioma', 'meningioma', 'pituitary'])

    # Obtenir les predictions du modèle
    predictions_encoded = model.predict(images)
    pred_class_names = [class_names[np.argmax(pred)] for pred in predictions_encoded]
    predicted_labels = multi_to_binary(pred_class_names, positif_classes=['glioma', 'meningioma', 'pituitary'])

    batch_len = min(num_samples, len(images), len(true_labels), len(predicted_labels))
    for i in range(batch_len):
        plt.subplot(rows, cols, i + 1)
        # Redimmensionner image pour affichage
        image = images[i] / 255.0
        plt.imshow(image)
        true_label = true_labels[i]
        predicted_label = predicted_labels[i]
        color = "green" if true_label == predicted_label else "red"
        plt.title(f"True: {true_label}\nPred: {predicted_label}", color=color)
        plt.axis("off")

    plt.tight_layout()
    plt.show()

try:
    model
except NameError:
    print("Erreur : 'model' n'est pas défini. Veuillez exécuter la cellule où le modèle est créé (cellule 21).")
else:
    predictions(model=model,
                test_generator=test_datagen,
                binary_labels=['Tumeur', 'Sain'],
                num_samples=16,
                figsize=(10, 10))